<h1 align='center' style='color:blue;'> Reconciliation <h1/>

#### Overview
Please understand the below mentioned real-life scenario and try to solve the assignment.The sample data is attached in the link provided below for your reference.<br>

#### Business Scenario
You are a data analyst and your client has a large ecommerce company in India (let’s call it X). X gets a thousand orders via their website on a daily basis and they have to deliver them as fast as they can. For delivering the goods ordered by the customers, X has tied up with multiple courier companies in India as delivery partners who charge them some amount per delivery.<br>

The charges are dependent upon two factors:
- Weight of the product
- Distance between the warehouse (pickup location) and customer’s delivery address (destination location)

On an average, the delivery charges are Rs. 100 per shipment. So if X ships 1,00,000 orders per month, they have to pay approximately Rs. 1 crore to the courier companies on a monthly basis as charges.<br>
As the amount that X has to pay to the courier companies is very high, they want to verify if the charges levied by their Delivery partners per Order are correct.

#### Input Data
Left Hand Side (LHS) Data (X’s internal data spread across three reports)
- Website order report- which will list Order IDs and various products (SKUs) part of each order. Order ID is common identifier between X’s order report and courier company invoice
- SKU master with gross weight of each product-This should be used to calculate total weight of each order and during analysis compare against one reported by courier company in their CSV invoice per Order ID. The courier company calculates weight in slabs of 0.5 KG multiples, so first you have to figure out the total weight of the shipment and then figure out applicable weight slabs.<br>
    For example:
    - If the total weight is 400 gram then weight slab should be 0.5
    - If the total weight is 950 gram then weight slab should be 1
    - If the total weight is 1 KG then weight slab should be 1
    - If the total weight is 2.2 KG then weight slab should be 2.5
- Warehouse pincode to All India pincode mapping -(this should be used to figure out delivery zone (a/b/c/d/e) and during analysis compare against one reported by courier company in their CSV invoice per Order ID

RHS Data (courier company invoice in CSV file)
- Invoice in CSV file mentioning AWB Number (courier company’s own internal ID), Order ID (company X’s order ID), weight of shipment, warehouse pickup pincode, customer delivery pincode, zone of delivery, charges per shipment, type of shipment
- Courier charges rate card at weight slab and pincode level. If the invoice mentions “Forward charges” then only forward charges (“fwd”) should be applicable as per zone and fixed & additional weights based on weight slabs. If the invoice mentions “Forward and rto charges” then forward charges (“fwd”) and RTO charges (“rto”) should be applicable as per zone and fixed & additional weights based on weight slabs.
- For the first 0.5 KG, “fixed” rate as per the slab is applicable. For each additional 0.5 KG, “additional” weight in the same proportion is applicable. Total charges will be “fixed” + “total additional” if any

#### Output Data 1
Create a resultant CSV/Excel file with the following columns:
- Order ID
- AWB Number
- Total weight as per X (KG)
- Weight slab as per X (KG)
- Total weight as per Courier Company (KG)
- Weight slab charged by Courier Company (KG)
- Delivery Zone as per X
- Delivery Zone charged by Courier Company
- Expected Charge as per X (Rs.)
- Charges Billed by Courier Company (Rs.)
- Difference Between Expected Charges and Billed Charges (Rs.)

#### Output Data 2
Create a summary table

|    | Count | Sum |
| --- | --- | --- |
| Total orders where X has been correctly charged | count | sum |
| Total orders where X has been over charged | count | sum |
| Total orders where X has been under charged | count | sum |



In [1]:
# importing libraries
import pandas as pd
import numpy as np
import math

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# loading all files
x_order = pd.read_excel('Company X - Order Report.xlsx')
x_pincode = pd.read_excel('Company X - Pincode Zones.xlsx')
x_sku = pd.read_excel('Company X - SKU Master.xlsx')
courier_invoice = pd.read_excel('Courier Company - Invoice.xlsx')
courier_rate = pd.read_excel('Courier Company - Rates.xlsx')

#### Analyse the Company X - Order Report

In [4]:
x_order.head()

,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0


In [5]:
x_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ExternOrderNo  400 non-null    int64  
 1   SKU            400 non-null    object 
 2   Order Qty      400 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 9.5+ KB


In [6]:
# # lookup the value of 'Weight (g)' from "x_sku" on the 'SKU'

x_order = pd.merge(x_order, 
                      x_sku, 
                      on ='SKU', 
                      how ='inner')
x_order

,ExternOrderNo,SKU,Order Qty,Weight (g)
0,2001827036,8904223818706,1.0,127
1,2001821995,8904223818706,1.0,127
2,2001819252,8904223818706,1.0,127
3,2001816996,8904223818706,1.0,127
4,2001814580,8904223818706,1.0,127
...,...,...,...,...
396,2001806616,8904223819123,1.0,250
397,2001806567,8904223815804,1.0,160
398,2001806567,8904223818577,1.0,150
399,2001806408,8904223819437,2.0,552


In [7]:
#calculating total weight in kg = (Weight (g)/1000)*Order Qty

x_order['Total Weight (kg)'] = (x_order['Weight (g)']/1000)*x_order['Order Qty']    

In [8]:
x_order

,ExternOrderNo,SKU,Order Qty,Weight (g),Total Weight (kg)
0,2001827036,8904223818706,1.0,127,0.127
1,2001821995,8904223818706,1.0,127,0.127
2,2001819252,8904223818706,1.0,127,0.127
3,2001816996,8904223818706,1.0,127,0.127
4,2001814580,8904223818706,1.0,127,0.127
...,...,...,...,...,...
396,2001806616,8904223819123,1.0,250,0.250
397,2001806567,8904223815804,1.0,160,0.160
398,2001806567,8904223818577,1.0,150,0.150
399,2001806408,8904223819437,2.0,552,1.104


- Before merging the dataframe total rows are 400
- After merging the dataframe total rows are 401
- Which means after merging 1 row increases
- **Objective**- find and remove additional row

In [9]:
#finding additional row using hit n trial 

x_order[x_order['ExternOrderNo']==2001827036]

,ExternOrderNo,SKU,Order Qty,Weight (g),Total Weight (kg)
0,2001827036,8904223818706,1.0,127,0.127
59,2001827036,8904223819093,1.0,150,0.150
60,2001827036,8904223819109,1.0,100,0.100
61,2001827036,8904223818430,1.0,165,0.165
66,2001827036,8904223819277,1.0,350,0.350
71,2001827036,GIFTBOX202002,1.0,500,0.500
72,2001827036,GIFTBOX202002,1.0,500,0.500
73,2001827036,8904223818638,2.0,137,0.274
77,2001827036,SACHETS001,1.0,10,0.010


- row 71 and 72 are identical
- removing row 72

In [10]:
# drop row 72

x_order.drop([72],inplace=True)

In [11]:
x_order

,ExternOrderNo,SKU,Order Qty,Weight (g),Total Weight (kg)
0,2001827036,8904223818706,1.0,127,0.127
1,2001821995,8904223818706,1.0,127,0.127
2,2001819252,8904223818706,1.0,127,0.127
3,2001816996,8904223818706,1.0,127,0.127
4,2001814580,8904223818706,1.0,127,0.127
...,...,...,...,...,...
396,2001806616,8904223819123,1.0,250,0.250
397,2001806567,8904223815804,1.0,160,0.160
398,2001806567,8904223818577,1.0,150,0.150
399,2001806408,8904223819437,2.0,552,1.104


In [12]:
# Creating dataframe that contain "Order ID" and "Total Weight (Kg)"

x_weight = pd.DataFrame(x_order.groupby('ExternOrderNo')['Total Weight (kg)'].sum())
x_weight=x_weight.reset_index()

In [13]:
# Rename the "ExternOrderNo" to "Order ID" because both are same

x_weight.rename(columns={'ExternOrderNo':'Order ID'},inplace=True)

In [14]:
x_weight.head()

,Order ID,Total Weight (kg)
0,2001806210,0.220
1,2001806226,0.480
2,2001806229,0.500
3,2001806232,1.302
4,2001806233,0.245


In [15]:
courier_invoice

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4
...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0


In [16]:
# Creating "result" dataframe

result = pd.DataFrame(courier_invoice[['Order ID','AWB Code']])

In [17]:
# lookup the value of 'Total Weight (kg)' from "x_weight" on the 'Order ID'

result = pd.merge(result, 
                      x_weight, 
                      on ='Order ID', 
                      how ='inner')
result.head()

,Order ID,AWB Code,Total Weight (kg)
0,2001806232,1091117222124,1.302
1,2001806273,1091117222194,0.615
2,2001806408,1091117222931,2.265
3,2001806458,1091117223244,0.700
4,2001807012,1091117229345,0.240


In [18]:
# Renaming column

result.rename(columns={'Total Weight (kg)':'Total Weight as per X (kg)'},inplace=True)
result.head()

,Order ID,AWB Code,Total Weight as per X (kg)
0,2001806232,1091117222124,1.302
1,2001806273,1091117222194,0.615
2,2001806408,1091117222931,2.265
3,2001806458,1091117223244,0.700
4,2001807012,1091117229345,0.240


In [19]:
# Creating column 'Weight slab as per X(kg)'
# Slab the Total Weight - round up to 0.5

result['Weight slab as per X(kg)'] = result['Total Weight as per X (kg)'].apply(lambda x : math.ceil(x*2)/2)
result.head()

,Order ID,AWB Code,Total Weight as per X (kg),Weight slab as per X(kg)
0,2001806232,1091117222124,1.302,1.5
1,2001806273,1091117222194,0.615,1.0
2,2001806408,1091117222931,2.265,2.5
3,2001806458,1091117223244,0.700,1.0
4,2001807012,1091117229345,0.240,0.5


In [20]:
# lookup the value of 'Charged Weight' from "courier_invoice" on the 'Order ID'

result = pd.merge(result, 
                      courier_invoice[['Order ID','Charged Weight']], 
                      on ='Order ID', 
                      how ='inner')

In [21]:
# renaming the column

result.rename(columns={'Charged Weight':'Total Weight as per courier company(kg)'},inplace=True)

In [22]:
result.head()

,Order ID,AWB Code,Total Weight as per X (kg),Weight slab as per X(kg),Total Weight as per courier company(kg)
0,2001806232,1091117222124,1.302,1.5,1.30
1,2001806273,1091117222194,0.615,1.0,1.00
2,2001806408,1091117222931,2.265,2.5,2.50
3,2001806458,1091117223244,0.700,1.0,1.00
4,2001807012,1091117229345,0.240,0.5,0.15


In [23]:
# Creating column 'Weight slab as per courier company (kg)'
# Slab the Total Weight - round up to 0.5

result['Weight slab as per courier company(kg)'] = result['Total Weight as per courier company(kg)'].apply(lambda x : math.ceil(x*2)/2)
result.head()

,Order ID,AWB Code,Total Weight as per X (kg),Weight slab as per X(kg),Total Weight as per courier company(kg),Weight slab as per courier company(kg)
0,2001806232,1091117222124,1.302,1.5,1.30,1.5
1,2001806273,1091117222194,0.615,1.0,1.00,1.0
2,2001806408,1091117222931,2.265,2.5,2.50,2.5
3,2001806458,1091117223244,0.700,1.0,1.00,1.0
4,2001807012,1091117229345,0.240,0.5,0.15,0.5


In [24]:
# lookup the value of 'zone' from "courier_invoice" on the 'Order ID'

result = pd.merge(result, 
                      courier_invoice[['Order ID','Zone']], 
                      on ='Order ID', 
                      how ='inner')

In [25]:
# Renaming column

result.rename(columns={'Zone':'Delivery zone as per X'},inplace=True)
result.head()

,Order ID,AWB Code,Total Weight as per X (kg),Weight slab as per X(kg),Total Weight as per courier company(kg),Weight slab as per courier company(kg),Delivery zone as per X
0,2001806232,1091117222124,1.302,1.5,1.30,1.5,d
1,2001806273,1091117222194,0.615,1.0,1.00,1.0,d
2,2001806408,1091117222931,2.265,2.5,2.50,2.5,d
3,2001806458,1091117223244,0.700,1.0,1.00,1.0,b
4,2001807012,1091117229345,0.240,0.5,0.15,0.5,d


In [26]:
# creating column 'Delivery zone charge by courier company'

result['Delivery zone charge by courier company'] = result['Delivery zone as per X']
result.head()

,Order ID,AWB Code,Total Weight as per X (kg),Weight slab as per X(kg),Total Weight as per courier company(kg),Weight slab as per courier company(kg),Delivery zone as per X,Delivery zone charge by courier company
0,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d
1,2001806273,1091117222194,0.615,1.0,1.00,1.0,d,d
2,2001806408,1091117222931,2.265,2.5,2.50,2.5,d,d
3,2001806458,1091117223244,0.700,1.0,1.00,1.0,b,b
4,2001807012,1091117229345,0.240,0.5,0.15,0.5,d,d


### Calcuation of 'Expected charges as per X'

In [27]:
# creating the dataframe having 'Order ID','Weight slab as per X(kg)','Delivery zone as per X'

calculation =pd.DataFrame(result[['Order ID','Weight slab as per X(kg)','Delivery zone as per X']])
calculation

,Order ID,Weight slab as per X(kg),Delivery zone as per X
0,2001806232,1.5,d
1,2001806273,1.0,d
2,2001806408,2.5,d
3,2001806458,1.0,b
4,2001807012,0.5,d
...,...,...,...
119,2001812941,0.5,d
120,2001809383,1.0,d
121,2001820978,1.0,d
122,2001811475,1.0,b


In [28]:
# lookup the value of 'Type of Shipment' from "courier_invoice" on the 'Order ID'

calculation = pd.merge(calculation, 
                      courier_invoice[['Order ID','Type of Shipment']], 
                      on ='Order ID', 
                      how ='inner')

calculation.head()

,Order ID,Weight slab as per X(kg),Delivery zone as per X,Type of Shipment
0,2001806232,1.5,d,Forward charges
1,2001806273,1.0,d,Forward charges
2,2001806408,2.5,d,Forward charges
3,2001806458,1.0,b,Forward charges
4,2001807012,0.5,d,Forward charges


In [29]:
courier_rate

,fwd_a_fixed,fwd_a_additional,fwd_b_fixed,fwd_b_additional,fwd_c_fixed,fwd_c_additional,fwd_d_fixed,fwd_d_additional,fwd_e_fixed,fwd_e_additional,rto_a_fixed,rto_a_additional,rto_b_fixed,rto_b_additional,rto_c_fixed,rto_c_additional,rto_d_fixed,rto_d_additional,rto_e_fixed,rto_e_additional
0,29.5,23.6,33,28.3,40.1,38.9,45.4,44.8,56.6,55.5,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5


In [30]:
# Creating a null column which will use in calculation of 'Expected charge as per X(Rs)'

calculation['Expected charge as per X(Rs)']=0

In [31]:
# Calculating 'Expected charge as per X(Rs)' using for loop
# logic
#      if zone=='d' and shipment=='Forward charges'
#          charge= fixed_charge + ((weight/0.5)-1)*additional_charge

    
for i in range(0,123):
    
    if ((calculation['Delivery zone as per X'][i]=='d') and (calculation['Type of Shipment'][i]=='Forward charges')):
        calculation['Expected charge as per X(Rs)'][i] = courier_rate['fwd_d_fixed'][0] + ((calculation['Weight slab as per X(kg)'][i]/0.5)-1)*courier_rate['fwd_d_additional'][0]
        
    elif ((calculation['Delivery zone as per X'][i]=='d') and (calculation['Type of Shipment'][i]=='Forward and RTO charges')):
        calculation['Expected charge as per X(Rs)'][i] = courier_rate['fwd_d_fixed'][0] + ((calculation['Weight slab as per X(kg)'][i]/0.5)-1)*courier_rate['fwd_d_additional'][0] + courier_rate['rto_d_fixed'][0] + ((calculation['Weight slab as per X(kg)'][i]/0.5)-1)*courier_rate['rto_d_additional'][0]
        
    elif ((calculation['Delivery zone as per X'][i]=='b') and (calculation['Type of Shipment'][i]=='Forward charges')):
        calculation['Expected charge as per X(Rs)'][i] = courier_rate['fwd_b_fixed'][0] + ((calculation['Weight slab as per X(kg)'][i]/0.5)-1)*courier_rate['fwd_b_additional'][0]
        
    elif ((calculation['Delivery zone as per X'][i]=='b') and (calculation['Type of Shipment'][i]=='Forward and RTO charges')):
        calculation['Expected charge as per X(Rs)'][i] = courier_rate['fwd_b_fixed'][0] + ((calculation['Weight slab as per X(kg)'][i]/0.5)-1)*courier_rate['fwd_b_additional'][0] + courier_rate['rto_b_fixed'][0] + ((calculation['Weight slab as per X(kg)'][i]/0.5)-1)*courier_rate['rto_b_additional'][0]
        
    elif ((calculation['Delivery zone as per X'][i]=='e') and (calculation['Type of Shipment'][i]=='Forward charges')):
        calculation['Expected charge as per X(Rs)'][i] = courier_rate['fwd_e_fixed'][0] + ((calculation['Weight slab as per X(kg)'][i]/0.5)-1)*courier_rate['fwd_e_additional'][0]
        
    elif ((calculation['Delivery zone as per X'][i]=='e') and (calculation['Type of Shipment'][i]=='Forward and RTO charges')):
        calculation['Expected charge as per X(Rs)'][i] = courier_rate['fwd_e_fixed'][0] + ((calculation['Weight slab as per X(kg)'][i]/0.5)-1)*courier_rate['fwd_e_additional'][0] + courier_rate['rto_e_fixed'][0] + ((calculation['Weight slab as per X(kg)'][i]/0.5)-1)*courier_rate['rto_e_additional'][0]
        
    calculation['Expected charge as per X(Rs)'].append(calculation['Expected charge as per X(Rs)'])

In [32]:
calculation['Expected charge as per X(Rs)']

0      135.0
1       90.2
2      224.6
3       61.3
4       45.4
       ...  
119     45.4
120    176.3
121     90.2
122     61.3
123      0.0
Name: Expected charge as per X(Rs), Length: 124, dtype: float64

In [33]:
calculation.head()

,Order ID,Weight slab as per X(kg),Delivery zone as per X,Type of Shipment,Expected charge as per X(Rs)
0,2001806232,1.5,d,Forward charges,135.0
1,2001806273,1.0,d,Forward charges,90.2
2,2001806408,2.5,d,Forward charges,224.6
3,2001806458,1.0,b,Forward charges,61.3
4,2001807012,0.5,d,Forward charges,45.4


In [34]:
# lookup the value of 'Expected charge as per X(Rs)' from "calculation" on the 'Order ID'

result = pd.merge(result, 
                      calculation[['Order ID','Expected charge as per X(Rs)']], 
                      on ='Order ID', 
                      how ='inner')

result.head()

,Order ID,AWB Code,Total Weight as per X (kg),Weight slab as per X(kg),Total Weight as per courier company(kg),Weight slab as per courier company(kg),Delivery zone as per X,Delivery zone charge by courier company,Expected charge as per X(Rs)
0,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,135.0
1,2001806273,1091117222194,0.615,1.0,1.00,1.0,d,d,90.2
2,2001806408,1091117222931,2.265,2.5,2.50,2.5,d,d,224.6
3,2001806458,1091117223244,0.700,1.0,1.00,1.0,b,b,61.3
4,2001807012,1091117229345,0.240,0.5,0.15,0.5,d,d,45.4


In [35]:
# lookup the value of 'Billing Amount (Rs.)' from "courier_invoice" on the 'Order ID'

result = pd.merge(result, 
                      courier_invoice[['Order ID','Billing Amount (Rs.)']], 
                      on ='Order ID', 
                      how ='inner')

In [36]:
# Renaming the column

result.rename(columns={'Billing Amount (Rs.)':'Charged billed by courier company(Rs)'},inplace=True)
result.head()

,Order ID,AWB Code,Total Weight as per X (kg),Weight slab as per X(kg),Total Weight as per courier company(kg),Weight slab as per courier company(kg),Delivery zone as per X,Delivery zone charge by courier company,Expected charge as per X(Rs),Charged billed by courier company(Rs)
0,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,135.0,135.0
1,2001806273,1091117222194,0.615,1.0,1.00,1.0,d,d,90.2,90.2
2,2001806408,1091117222931,2.265,2.5,2.50,2.5,d,d,224.6,224.6
3,2001806458,1091117223244,0.700,1.0,1.00,1.0,b,b,61.3,61.3
4,2001807012,1091117229345,0.240,0.5,0.15,0.5,d,d,45.4,45.4


In [37]:
# Creating the column 'Difference between expected charges and billed charges(Rs)'

result['Difference between expected charges and billed charges(Rs)'] = round(result['Expected charge as per X(Rs)']-result['Charged billed by courier company(Rs)'],3)

In [38]:
result

,Order ID,AWB Code,Total Weight as per X (kg),Weight slab as per X(kg),Total Weight as per courier company(kg),Weight slab as per courier company(kg),Delivery zone as per X,Delivery zone charge by courier company,Expected charge as per X(Rs),Charged billed by courier company(Rs),Difference between expected charges and billed charges(Rs)
0,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,135.0,135.0,0.0
1,2001806273,1091117222194,0.615,1.0,1.00,1.0,d,d,90.2,90.2,-0.0
2,2001806408,1091117222931,2.265,2.5,2.50,2.5,d,d,224.6,224.6,0.0
3,2001806458,1091117223244,0.700,1.0,1.00,1.0,b,b,61.3,61.3,0.0
4,2001807012,1091117229345,0.240,0.5,0.15,0.5,d,d,45.4,45.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...
119,2001812941,1091118551656,0.500,0.5,0.73,1.0,d,d,45.4,90.2,-44.8
120,2001809383,1091117614452,0.607,1.0,0.50,0.5,d,d,176.3,86.7,89.6
121,2001820978,1091120922803,0.515,1.0,0.50,0.5,d,d,90.2,45.4,44.8
122,2001811475,1091121844806,0.689,1.0,0.50,0.5,b,b,61.3,33.0,28.3


In [39]:
# storing into the excel file
result.to_excel("Results.xlsx")

In [40]:
# Count of Total Orders - Correctly Charged 

cc=result[result['Difference between expected charges and billed charges(Rs)']==0].count()['Difference between expected charges and billed charges(Rs)']
cc

53

In [41]:
# Count of Total Orders - Over Charged

co=result[result['Difference between expected charges and billed charges(Rs)']<0].count()['Difference between expected charges and billed charges(Rs)']
co

52

In [42]:
# Count of Total Orders - Under Charged

cu=result[result['Difference between expected charges and billed charges(Rs)']>0].count()['Difference between expected charges and billed charges(Rs)']
cu

19

In [43]:
# sum of Total Orders - Correctly Charged

sc=round(result[result['Difference between expected charges and billed charges(Rs)']==0].sum()['Expected charge as per X(Rs)'],2)
sc

4602.5

In [44]:
# sum of Total Orders - Over Charged

so=round(result[result['Difference between expected charges and billed charges(Rs)']<0].sum()['Expected charge as per X(Rs)'],2)
so

2765.6

In [45]:
# sum of Total Orders - Under Charged

su=round(result[result['Difference between expected charges and billed charges(Rs)']>0].sum()['Expected charge as per X(Rs)'],2)
su

3517.8

In [46]:
data = {'Count': [cc,co,cu],
        'Sum': [sc,so,su]}

In [47]:
Summary = pd.DataFrame(data, index=['Total Orders - Correctly Charged','Total Orders - Over Charged','Total Orders - Under Charged'])
Summary

,Count,Sum
Total Orders - Correctly Charged,53,4602.5
Total Orders - Over Charged,52,2765.6
Total Orders - Under Charged,19,3517.8


In [48]:
# storing into the excel file
Summary.to_excel("summary.xlsx")